In [1]:
import sys
from kaitoupao_wsl import *
# from kaitoupao import *

storage dir: /mnt/d/forCoding_data/Tianchi_NLPNewsClassification
code dir: /mnt/d/forCoding_code/Tianchi_NLPNewsClassification 

16 02 34
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：8 本卦下：2 变爻：4


,地泽临,地雷复,雷泽归妹
上卦,☷坤土,☷坤土,☳震木
下卦,☱兑金,☳震木,☱兑金


02 17 2 丑时
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：2 本卦下：1 变爻：2


,泽天夬,天天乾,泽火革
上卦,☱兑金,☰乾金,☱兑金
下卦,☰乾金,☰乾金,☲离火


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import f1_score

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device("cuda")
# device = torch.device("mps")

In [5]:
scheme_type = "20250221_2" 

# 加载训练集和测试集，将全量字符列表给它弄出来

In [6]:
data_train = pd.read_csv(create_originalData_path("train_set.csv"), sep="\t", nrows=None)#.sample(1000) , nrows=1000
data_test = pd.read_csv(create_originalData_path("test_a.csv"), sep="\t", nrows=None)#.sample(1000)

In [7]:
data_train.shape, data_test.shape

((200000, 2), (50000, 1))

In [8]:
type_of_class = data_train.label.nunique()

In [9]:
type_of_class

14

In [10]:
train_data, valid_data = train_test_split(data_train, test_size=0.3, random_state=42)

In [11]:
train_data = train_data.reset_index(drop=True)
valid_data = valid_data.reset_index(drop=True)

In [12]:
train_labels = torch.tensor(train_data.label.to_list(), dtype=torch.long)
valid_labels = torch.tensor(valid_data.label.to_list(), dtype=torch.long)
test_labels = torch.tensor([-1 for x in range(data_test.shape[0])], dtype=torch.long) ## test_labels的label是假的。

In [13]:
# data_train.text.to_list()[1].replace(" ", ",")

## 使用TF-IDF提取特征

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_train_features = tfidf_vectorizer.fit_transform(train_data['text'])
tfidf_valid_features = tfidf_vectorizer.transform(valid_data['text'])
tfidf_test_features = tfidf_vectorizer.transform(data_test['text'])

In [15]:
train_features = torch.tensor(tfidf_train_features.toarray(), dtype=torch.float32)
valid_features = torch.tensor(tfidf_valid_features.toarray(), dtype=torch.float32)
test_features = torch.tensor(tfidf_test_features.toarray(), dtype=torch.float32)

In [16]:
sc_input_dim = train_features.shape[1]
sc_input_dim

6695

## 准备数据集，tokenizer 

In [17]:
# 下载数据并进行预处理
vocab_size = 8000  # 只考虑前 20k 词汇
maxlen = 800  # 只考虑每条评论的前 800 个词

In [18]:
save_feaList_to_file([str(i) for i in range(vocab_size)], "./vocab.txt")

Finished writing file: ./vocab.txt


In [19]:
from transformers import BertTokenizer
tknz = BertTokenizer(
    vocab_file="vocab.txt",
    model_max_length=maxlen
)
vocab_size += len(tknz.all_special_tokens) ###### 这里，我们需要改一下vocab_size。不知道为什么，在mps上跑，都不用改这个，但是在cpu和cuda上跑就要改。诡异了。
tknz

/home/xiuminke/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertTokenizer(name_or_path='', vocab_size=8000, model_max_length=800, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	8000: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	8001: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	8002: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	8003: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	8004: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [20]:
batchsize = 32

In [21]:
class MyData(Dataset):
    def __init__(
        self, 
        ori_data, tfidf_feats, label,
    ):
        self.ori_data = ori_data
        self.tfidf_feats = tfidf_feats
        self.label = label
        self.mlm_prob = 0.15
        self.tokenizer = tknz
        self.max_len = maxlen
 
    def __len__(self):
        return len(self.ori_data)
 
    def __getitem__(self, idx):
        # tuple_ = (
        #     self.ori_data[idx], 
        #     self.tfidf_feats[idx], 
        #     self.label[idx]
        # )
        # return tuple_

        text = self.ori_data[idx]
        label = self.label[idx]
        
        # 1. 原始文本编码（用于分类）
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # 2. 生成MLM数据
        input_ids = encoding["input_ids"].squeeze(0)
        mlm_input_ids = input_ids.clone()
        mlm_labels = torch.full_like(input_ids, -100)  # 默认忽略非mask位置
        
        # 随机选择15%的token进行mask
        rand = torch.rand(input_ids.shape)
        mask_indices = (rand < self.mlm_prob) & (input_ids != self.tokenizer.cls_token_id) & (input_ids != self.tokenizer.sep_token_id)
        
        # 80%替换为[MASK], 10%随机词, 10%保持原词
        replace_mask = mask_indices & (torch.rand(mask_indices.shape) < 0.8)
        random_mask = mask_indices & (torch.rand(mask_indices.shape) < 0.5) & ~replace_mask
        
        mlm_input_ids[replace_mask] = self.tokenizer.mask_token_id
        mlm_input_ids[random_mask] = torch.randint(0, self.tokenizer.vocab_size, (sum(random_mask),))
        mlm_labels[mask_indices] = input_ids[mask_indices]

        # 3. 生成SOP数据（句子顺序预测）
        sentences = text.split('.')  # 简单按句号分割
        if len(sentences) >= 2:
            # 50%概率交换前两句
            if torch.rand(1) < 0.5:
                sent1, sent2 = sentences[0], sentences[1]
                sop_label = 1  # 顺序正确
            else:
                sent1, sent2 = sentences[1], sentences[0]
                sop_label = 0  # 顺序错误
            
            sop_text = f"{sent1} [SEP] {sent2}"
            sop_encoding = self.tokenizer(
                sop_text,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            sop_input_ids = sop_encoding["input_ids"].squeeze(0)
            sop_attention_mask = sop_encoding["attention_mask"].squeeze(0)
        else:
            sop_input_ids = input_ids
            sop_attention_mask = encoding["attention_mask"].squeeze(0)
            sop_label = -100  # 忽略此样本

        return (
            {
                "input_ids": input_ids,
                "attention_mask": encoding["attention_mask"].squeeze(0),
                "labels": torch.tensor(label, dtype=torch.long),
                "mlm_input_ids": mlm_input_ids,
                "mlm_labels": mlm_labels,
                "sop_input_ids": sop_input_ids,
                "sop_attention_mask": sop_attention_mask,
                "sop_labels": torch.tensor(sop_label, dtype=torch.long)
            }, 
            self.tfidf_feats[idx],
            label,
       )
        

In [22]:
train_loader = DataLoader(MyData(train_data.text, train_features, train_labels), batch_size=batchsize, shuffle=True,)
val_loader = DataLoader(MyData(valid_data.text, valid_features, valid_labels), batch_size=batchsize, shuffle=True,)
test_loader = DataLoader(MyData(data_test.text, test_features, test_labels), batch_size=batchsize, shuffle=False,) ## 这个不要shuffle，否则传到oj上面去就GG了。

# 定义网络结构

In [23]:
# 定义联合训练模型
class JointModel(nn.Module):
    def __init__(self, bert_model_name, num_labels, mlm_vocab_size=vocab_size):
        super().__init__()
        # 共享的BERT编码器
        self.bert = BertModel(BertConfig(
                vocab_size=vocab_size, ## 
                hidden_size=32,
                num_hidden_layers=4,
                num_attention_heads=2,
                intermediate_size=64,
                max_position_embeddings=maxlen,
                num_labels=num_labels
            ))
        self.config = self.bert.config
        
        # 分类任务头
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)
        
        # MLM任务头
        self.mlm_head = nn.Linear(self.config.hidden_size, mlm_vocab_size)
        
        # SOP任务头
        self.sop_head = nn.Linear(self.config.hidden_size, 2)  # 二分类：顺序是否正确

    def forward(self, input_ids, attention_mask, token_type_ids=None, 
                mlm_labels=None, sop_labels=None):
        # 共享编码器输出
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True
        )
        # print(outputs)
        sequence_output = outputs.last_hidden_state  # [batch, seq_len, hidden]
        pooled_output = outputs.pooler_output        # [batch, hidden]

        # 分类任务
        cls_logits = self.classifier(pooled_output)  # [batch, num_labels]

        # MLM任务
        mlm_logits = self.mlm_head(sequence_output)  # [batch, seq_len, vocab]
        # print(mlm_logits.shape)

        # SOP任务
        sop_logits = self.sop_head(pooled_output)    # [batch, 2]

        # 计算各任务损失
        losses = {}
        if mlm_labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            mlm_loss = loss_fct(
                mlm_logits.view(-1, self.config.vocab_size),
                mlm_labels.view(-1)
            )
            losses["mlm"] = mlm_loss

        if sop_labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            sop_loss = loss_fct(sop_logits.view(-1, 2), sop_labels.view(-1))
            losses["sop"] = sop_loss

        return cls_logits, losses

# 构建模型以及训练

In [24]:
from transformers import BertConfig
from transformers import BertForSequenceClassification, BertModel

In [25]:
model = JointModel("", type_of_class)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [26]:
model

JointModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8005, 32, padding_idx=0)
      (position_embeddings): Embedding(800, 32)
      (token_type_embeddings): Embedding(2, 32)
      (LayerNorm): LayerNorm((32,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=32, out_features=32, bias=True)
              (key): Linear(in_features=32, out_features=32, bias=True)
              (value): Linear(in_features=32, out_features=32, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=32, out_features=32, bias=True)
              (LayerNorm): LayerNorm((32,), eps=1e-12, elementwise_affine=True)
     

In [27]:
from torch.utils.tensorboard import SummaryWriter
import shutil

# 训练和评估模型
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2):

    ## loss_weights仅用于无监督预训练场景：
    loss_weights={'cls': 1.0, 'mlm': 0.5, 'sop': 0.5}
    
    model.to(device)
    
    log_path = "runs/Logs"
    if os.path.exists(
        os.path.dirname(log_path)
    ):
        shutil.rmtree(os.path.dirname(log_path))
        
    writer = SummaryWriter(log_dir=log_path)
    
    total_train_step = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        loss_sum = 0
        for batch, input_tfidfs, targets in tqdm.tqdm(train_loader):
            input_tfidfs, targets = input_tfidfs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            # 分类任务
            cls_output, _ = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device)
            )
            cls_loss = nn.CrossEntropyLoss()(cls_output, batch['labels'].to(device))
            # MLM任务
            _, mlm_losses = model(
                input_ids=batch['mlm_input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                mlm_labels=batch['mlm_labels'].to(device)
            )
            # SOP任务
            _, sop_losses = model(
                input_ids=batch['sop_input_ids'].to(device),
                attention_mask=batch['sop_attention_mask'].to(device),
                sop_labels=batch['sop_labels'].to(device)
            )
            # 加权总损失
            total_loss = (
                loss_weights['cls'] * cls_loss +
                loss_weights['mlm'] * mlm_losses['mlm'] +
                loss_weights['sop'] * sop_losses['sop']
            )
            total_loss.backward()
            optimizer.step()
            
            train_loss += total_loss.item()
            _, predicted = torch.max(cls_output, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            total_train_step += 1
            loss_sum += total_loss.item()
            if total_train_step % 10 == 0:
                # print(total_train_step//10)
                writer.add_scalar(f"train_loss_detail-{scheme_type}", loss_sum/10, total_train_step//10)
                loss_sum = 0
                
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss/len(train_loader)}, Accuracy: {100.*correct/total}%')
        writer.add_scalar(f"epoch_loss-{scheme_type}", train_loss/len(train_loader), epoch+1)
        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            total_predicted = []
            total_label = []
            for batch, input_tfidfs, targets in tqdm.tqdm(val_loader):
                input_tfidfs, targets = input_tfidfs.to(device), targets.to(device)
                outputs, _ = model(
                    input_ids=batch['input_ids'].to(device),
                    attention_mask=batch['attention_mask'].to(device)
                )
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                total_predicted += list(predicted.cpu())
                total_label += list(targets.cpu())
        f1 = f1_score(total_label, total_predicted, average='macro')
                
        print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100.*correct/total}%, f1 score is {f1}')
    writer.close()

In [28]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

  0%|                                                                                          | 0/4375 [00:00<?, ?it/s]/tmp/ipykernel_6121/3999588957.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label, dtype=torch.long),
 88%|█████████████████████████████████████████████████████████████████████▏         | 3833/4375 [19:21<02:44,  3.30it/s]


KeyboardInterrupt: 

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=4)

In [29]:
save_pickle_object(model, create_trained_models_path(f"ori_9319-{scheme_type}-rnd1.pkl"))

/mnt/d/forCoding_data/Tianchi_NLPNewsClassification/trained_models/ori_9319-20250221_2-rnd1.pkl


In [ ]:
## 后续可以尝试，先对其他的任务进行预训练，然后再训练分类任务。

In [ ]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1) 

In [ ]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

In [ ]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

In [ ]:
# run_finish()

# 保存模型

In [ ]:
save_pickle_object(model, create_trained_models_path(f"ori_9319-{scheme_type}.pkl"))

# 预测一下试试

In [ ]:
# model = load_pickle_object(create_trained_models_path(f"ori_9319-{scheme_type}.pkl"))

In [ ]:
with torch.no_grad():
    total_predicted = []
    for batch, input_tfidfs, targets in tqdm.tqdm(test_loader):
        input_tfidfs, targets = input_tfidfs.to(device), targets.to(device)
        outputs, _ = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device)
        )
        _, predicted = torch.max(outputs, 1)
        total_predicted += list(predicted)

In [ ]:
oot_rst = [int(x) for x in total_predicted]
len(oot_rst)

In [ ]:
store_data_to_newbasepath(pd.DataFrame({"label": oot_rst}), f"rst-{scheme_type}", fmt="csv")